### Initial settings

##### Libraries

In [1]:
# ----- Load libraries -----

using Dates;
using JLD;
using Distributions;
using DataFrames;
using Colors;

# PlotlyJS
using PlotlyJS;
using ORCA;
pltjs = PlotlyJS

include("./code/JuSSM/JuSSM.jl");
using Main.JuSSM;

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-8529438378388191049\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9711201932641018616\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-8529438378388191049\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

##### Insert manual input

In [2]:
# File path
model_folder = "./models/baseline_cond_2020/";
res_iis_name = "_iis_2019" 

# true for baseline, false for restricted
is_baseline = true;

##### Initial settings

In [3]:
# ----- Load results from output file -----

# In-sample output
res_iis       = jldopen("$(model_folder)/results/res$(res_iis_name).jld");
nDraws        = read(res_iis, "nDraws");
burnin        = read(res_iis, "burnin");
σʸ            = permutedims(read(res_iis, "σʸ"));
date          = read(res_iis, "date");
chain_θ_bound = read(res_iis, "chain_θ_bound"); # d, Z, R, c, T, Q, λ, ρ, total
MNEMONIC      = read(res_iis, "MNEMONIC");

par_ind  = read(res_iis, "par_ind");

par_size = SizeParSsm(sum(par_ind.d),
                                 sum(sum(par_ind.Z)),
                                 sum(sum(par_ind.R)),
                                 sum(par_ind.c),
                                 sum(sum(par_ind.T)),
                                 sum(sum(par_ind.Q)),
                                 sum(sum(par_ind.λ)),
                                 sum(sum(par_ind.ρ)),
                                 sum(par_ind.d) + sum(  sum(par_ind.Z)) + sum(sum(par_ind.R)) +
                                    sum(par_ind.c) + sum(sum(par_ind.T)) + sum(sum(par_ind.Q)) +
                                    sum(sum(par_ind.λ)) + sum(sum(par_ind.ρ)));

# Conditional forecast output
res        = jldopen("$(model_folder)/results/res.jld");
data       = read(res, "data_cond");
distr_α    = read(res, "distr_cond_α");
distr_fcst = read(res, "distr_cond_fcst");
data[ismissing.(data)] .= NaN;

# Offset
restricted_offset = is_baseline == false |> Int64;

In [4]:
# ----- Set titles, scales and idiosyncratic cycles and trends IDs -----

n  = size(data)[2];
TT = size(distr_α, 2);

# Set trends and cycles id
ind_trends = [10; collect(15:3:size(distr_α)[1])[1:3]; 28] # There is no index for inflation, please use if-else
ind_cycles = [8; 11; collect(13:3:size(distr_α)[1])[1:4]; 24; 26];

# Set titles and scales
titles = ["Real GDP", "SPF: Expected GDP", "Unemployment rate", "Employment", "Oil price", "CPI inflation", "SPF: Expected inflation", "UoM: Expected inflation"];
scales = ["Bil. Chn. 2009\$", "Bil. Chn. 2009\$", "Percent", "Thous.", "\$/Barrel", "Percent", "Percent", "Percent"];

if is_baseline == false
    
    # Set titles and scales
    titles = vcat("CBO: Real GDP cycle", titles);
    scales = vcat("Bil. Chn. 2009\$", scales); # The first two entries are equal (that's fine)
    
    # Set trends and cycles id
    ind_cycles = vcat(8, ind_cycles); # The first two entries are equal (that's fine)
end

In [5]:
#=
The first coefficient in chain_θ_bound is the intercept (differencial) for GDP SPF
Thus, starting from id 2 is correct
=#

Zᵖ = [ones(1, nDraws-burnin); 
      chain_θ_bound[2:4, burnin+1:end];
      zeros(1, nDraws-burnin);
      chain_θ_bound[5:7, burnin+1:end]];
 
Zᵖ₂ = [zeros(1, nDraws-burnin); 
       chain_θ_bound[8:10, burnin+1:end];
       zeros(1, nDraws-burnin);
       chain_θ_bound[11:13, burnin+1:end]];

Zᵖ₃ = [zeros(1, nDraws-burnin); 
       chain_θ_bound[14:16, burnin+1:end];
       zeros(1, nDraws-burnin);
       chain_θ_bound[17:19, burnin+1:end]];

Zᵖ₄ = [zeros(1, nDraws-burnin); 
       chain_θ_bound[20:22, burnin+1:end];
       zeros(1, nDraws-burnin);
       chain_θ_bound[23:25, burnin+1:end]];

Zᵉ = [zeros(4, nDraws-burnin); 
      ones(1, nDraws-burnin); 
      chain_θ_bound[26:28, burnin+1:end]];

Zᵗ = [zeros(5, nDraws-burnin); 
      (1 ./ σʸ[end-2:end]).*ones(3, nDraws-burnin)];

Z_GDP = [([1;3] ./ σʸ[[1,2] .+ restricted_offset]).*ones(2, nDraws-burnin);
         zeros(6, nDraws-burnin)];

if is_baseline == false

    # Add loadings for the "CBO: Real GDP cycle"
    Zᵖ    = vcat(ones(1, nDraws-burnin), Zᵖ);
    Zᵖ₂   = vcat(zeros(1, nDraws-burnin), Zᵖ₂);
    Zᵖ₃   = vcat(zeros(1, nDraws-burnin), Zᵖ₃);
    Zᵖ₄   = vcat(zeros(1, nDraws-burnin), Zᵖ₄);
    Zᵉ    = vcat(zeros(1, nDraws-burnin), Zᵉ);
    Zᵗ    = vcat(zeros(1, nDraws-burnin), Zᵗ);
    Z_GDP = vcat(zeros(1, nDraws-burnin), Z_GDP);
end

##### Collect states and compute quantiles

In [6]:
PC = zeros(n, TT, nDraws-burnin);
EP = zeros(n, TT, nDraws-burnin);
T  = zeros(n, TT, nDraws-burnin);
T_GDP = zeros(n, TT, nDraws-burnin);

for i=1:nDraws-burnin   
    PC[:, :, i] = (Zᵖ[:, i] .* distr_α[1, :, i]') .+ 
                  (Zᵖ₂[:, i] .* distr_α[2, :, i]') .+
                  (Zᵖ₃[:, i] .* distr_α[3, :, i]') .+
                  (Zᵖ₄[:, i] .* distr_α[4, :, i]');
    
    EP[:, :, i] = Zᵉ[:, i] .* distr_α[5, :, i]';
    T[:, :, i]  = Zᵗ[:, i] .* distr_α[7, :, i]';
    T_GDP[:, :, i] = Z_GDP[:, i] .* distr_α[10, :, i]';    
    T_GDP[2 + restricted_offset, :, i] .+= chain_θ_bound[1,burnin+i];
end

In [7]:
# Collect distributions of idiosyncratic components
iC = distr_α[ind_cycles, :, :];
iT = distr_α[ind_trends, :, :];

# Compute medians
PCᵐ    = median(PC, dims=3)[:, :, 1]' .* σʸ;
EPᵐ    = median(EP, dims=3)[:, :, 1]' .* σʸ;
Tᵐ     = median(T, dims=3)[:, :, 1]' .* σʸ;
T_GDPᵐ = median(T_GDP, dims=3)[:, :, 1]' .* σʸ;
iCᵐ    = median(iC, dims=3)[:, :, 1]' .* σʸ;
iTᵐ    = median(iT, dims=3)[:, :, 1]' .* σʸ[[1,3,4,5,8] .+ restricted_offset]';

In [8]:
# Compute quantiles

PC95 = zeros(size(PCᵐ));
PC05 = zeros(size(PCᵐ));
PC84 = zeros(size(PCᵐ));
PC16 = zeros(size(PCᵐ));

EP95 = zeros(size(EPᵐ));
EP05 = zeros(size(EPᵐ));
EP84 = zeros(size(EPᵐ));
EP16 = zeros(size(EPᵐ));

T95 = zeros(size(Tᵐ));
T05 = zeros(size(Tᵐ));
T84 = zeros(size(Tᵐ));
T16 = zeros(size(Tᵐ));

T_GDP95 = zeros(size(T_GDPᵐ));
T_GDP05 = zeros(size(T_GDPᵐ));
T_GDP84 = zeros(size(T_GDPᵐ));
T_GDP16 = zeros(size(T_GDPᵐ));

iC95 = zeros(size(iCᵐ));
iC05 = zeros(size(iCᵐ));
iC84 = zeros(size(iCᵐ));
iC16 = zeros(size(iCᵐ));

iT95 = zeros(size(iTᵐ));
iT05 = zeros(size(iTᵐ));
iT84 = zeros(size(iTᵐ));
iT16 = zeros(size(iTᵐ));
iTᵐ[:, 1] = copy(T_GDPᵐ[:, 1 + restricted_offset]);

for i=1:size(data, 2)
    for j=1:size(data, 1)
        
        PC95[j, i] = quantile(PC[i, j, :], 0.95) .* σʸ[i];
        PC05[j, i] = quantile(PC[i, j, :], 0.05) .* σʸ[i];
        PC84[j, i] = quantile(PC[i, j, :], 0.84) .* σʸ[i];
        PC16[j, i] = quantile(PC[i, j, :], 0.16) .* σʸ[i];
        
        EP95[j, i] = quantile(EP[i, j, :], 0.95) .* σʸ[i];
        EP05[j, i] = quantile(EP[i, j, :], 0.05) .* σʸ[i];
        EP84[j, i] = quantile(EP[i, j, :], 0.84) .* σʸ[i];
        EP16[j, i] = quantile(EP[i, j, :], 0.16) .* σʸ[i];

        T95[j, i] = quantile(T[i, j, :], 0.95) .* σʸ[i];
        T05[j, i] = quantile(T[i, j, :], 0.05) .* σʸ[i];
        T84[j, i] = quantile(T[i, j, :], 0.84) .* σʸ[i];
        T16[j, i] = quantile(T[i, j, :], 0.16) .* σʸ[i];

        T_GDP95[j, i] = quantile(T_GDP[i, j, :], 0.95) .* σʸ[i];
        T_GDP05[j, i] = quantile(T_GDP[i, j, :], 0.05) .* σʸ[i];
        T_GDP84[j, i] = quantile(T_GDP[i, j, :], 0.84) .* σʸ[i];
        T_GDP16[j, i] = quantile(T_GDP[i, j, :], 0.16) .* σʸ[i];

        iC95[j, i] = quantile(iC[i, j, :], 0.95) .* σʸ[i];
        iC05[j, i] = quantile(iC[i, j, :], 0.05) .* σʸ[i];
        iC84[j, i] = quantile(iC[i, j, :], 0.84) .* σʸ[i];
        iC16[j, i] = quantile(iC[i, j, :], 0.16) .* σʸ[i];
        
        if i == 1 + restricted_offset
            iT95[j, i-restricted_offset] = copy(T_GDP95[j, i]);
            iT05[j, i-restricted_offset] = copy(T_GDP05[j, i]);
            iT84[j, i-restricted_offset] = copy(T_GDP84[j, i]);
            iT16[j, i-restricted_offset] = copy(T_GDP16[j, i]);
            
        elseif i > 2 + restricted_offset && i <= 5 + restricted_offset
            iT95[j, i-1-restricted_offset] = quantile(iT[i-1-restricted_offset, j, :], 0.95) .* σʸ[i];
            iT05[j, i-1-restricted_offset] = quantile(iT[i-1-restricted_offset, j, :], 0.05) .* σʸ[i];
            iT84[j, i-1-restricted_offset] = quantile(iT[i-1-restricted_offset, j, :], 0.84) .* σʸ[i];
            iT16[j, i-1-restricted_offset] = quantile(iT[i-1-restricted_offset, j, :], 0.16) .* σʸ[i];
            
        elseif i == 8 + restricted_offset
            iT95[j, i-3-restricted_offset] = quantile(iT[i-3-restricted_offset, j, :], 0.95) .* σʸ[i];
            iT05[j, i-3-restricted_offset] = quantile(iT[i-3-restricted_offset, j, :], 0.05) .* σʸ[i];
            iT84[j, i-3-restricted_offset] = quantile(iT[i-3-restricted_offset, j, :], 0.84) .* σʸ[i];
            iT16[j, i-3-restricted_offset] = quantile(iT[i-3-restricted_offset, j, :], 0.16) .* σʸ[i];
        end        
    end
end

### Supplementary settings

In [9]:
# ----- Dates: add h dates to date -----

max_h = size(PCᵐ)[1] - size(date)[1];

for hz=1:max_h
    
    last_month = Dates.month(date[end]);
    last_year  = Dates.year(date[end]);
    new_month  = copy(last_month) + 1;
    new_year   = copy(last_year);
    
    if last_month + 1 > 12
        new_year  += 1;
        new_month  = 1;
    end
    
    new_entry = Dates.lastdayofquarter(Date(new_year, new_month, 1));
    date      = vcat(date, DateTime(new_entry));
end

last_observations = [findall(.~isnan.(data[:, i]))[end] for i=1:size(data,2)];

### Charts

##### Historical decompositions

In [10]:
figures = Array{Any}(undef, size(data, 2));

c1 = "rgba(0, 48, 158, .75)"; #"rgba(0, 0, 158, .7)";
c2 = "rgba(255, 0, 0, .75)";
c3 = "rgba(255, 190, 0, .75)";

for i=1:size(data, 2)
    trace1 = pltjs.bar(;x=date[1:last_observations[i]], y=PCᵐ[1:last_observations[i], i], name="Business cycle", marker_color=c1, showlegend=i==1);
    trace2 = pltjs.bar(x=date[1:last_observations[i]], y=EPᵐ[1:last_observations[i], i], name="Energy price cycle", marker_color=c2, showlegend=i==1);
    trace3 = pltjs.bar(x=date[1:last_observations[i]], y=iCᵐ[1:last_observations[i], i], name="Idiosyncratic cycle", marker_color=c3, showlegend=i==1);
    trace4 = pltjs.scatter(x=date[1:last_observations[i]], y=PCᵐ[1:last_observations[i], i]+EPᵐ[1:last_observations[i], i]+iCᵐ[1:last_observations[i], i], name="Total cycle", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==1)
    
    databar = [trace1, trace2, trace3, trace4];
    
        layout  = pltjs.Layout(;title=titles[i], titlefont_size=10,
                               xaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=-90),
                               yaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, titlefont=attr(size=14), title=scales[i]),
                               barmode="relative", 
                               bargap=0,
                               bargroupgap=0);
        

    figures[i] = pltjs.plot(databar, layout);
end

if is_baseline == true
    fig = [figures[1] figures[2]; figures[3] figures[4]; figures[5] figures[6]; figures[7] figures[8]];
else
    fig = [figures[1] figures[2] figures[3]; figures[4] figures[5] figures[6]; figures[7] figures[8] figures[9]];
end
    
# Bars
fig.plot.layout["barmode"] = "relative";
fig.plot.layout["bargap"]  = 0.02;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:size(data, 2)
    fig.plot.layout["annotations"][i][:font][:size] = 20;
end

# Legend
if is_baseline == true

    # Size
    fig.plot.layout["width"]  = 800;
    fig.plot.layout["height"] = 1000;
    fig.plot.layout["legend"] = attr(orientation="h", y=-0.08, x=0.065, font=attr(size=14))

else
    # Size
    fig.plot.layout["width"]  = 1000;
    fig.plot.layout["height"] = 800;
    fig.plot.layout["legend"] = attr(orientation="h", y=-0.08, x=0.065, font=attr(size=14))
end

pltjs.savefig(fig, "$(model_folder)/img/historical_decomposition.pdf", format="pdf")

"./models/baseline_iis//img/historical_decomposition.pdf"

In [11]:
figures = Array{Any}(undef, size(data, 2));

c1 = "rgba(0, 48, 158, .75)"; #"rgba(0, 0, 158, .7)";
c2 = "rgba(255, 0, 0, .75)";
c3 = "rgba(255, 190, 0, .75)";

for i=1:size(data, 2)
    trace1 = pltjs.bar(;x=date[1:end], y=PCᵐ[1:end, i], name="Business cycle", marker_color=c1, showlegend=i==1);
    trace2 = pltjs.bar(x=date[1:end], y=EPᵐ[1:end, i], name="Energy price cycle", marker_color=c2, showlegend=i==1);
    trace3 = pltjs.bar(x=date[1:end], y=iCᵐ[1:end, i], name="Idiosyncratic cycle", marker_color=c3, showlegend=i==1);
    trace4 = pltjs.scatter(x=date[1:end], y=PCᵐ[1:end, i]+EPᵐ[1:end, i]+iCᵐ[1:end, i], name="Total cycle", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==1)
    
    databar = [trace1, trace2, trace3, trace4];
    
        layout  = pltjs.Layout(;title=titles[i], titlefont_size=10,
                               xaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=-90),
                               yaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, titlefont=attr(size=14), title=scales[i]),
                               barmode="relative", 
                               bargap=0,
                               bargroupgap=0);
        

    figures[i] = pltjs.plot(databar, layout);
end

if is_baseline == true
    fig = [figures[1] figures[2]; figures[3] figures[4]; figures[5] figures[6]; figures[7] figures[8]];
else
    fig = [figures[1] figures[2] figures[3]; figures[4] figures[5] figures[6]; figures[7] figures[8] figures[9]];
end
    
# Bars
fig.plot.layout["barmode"] = "relative";
fig.plot.layout["bargap"]  = 0.02;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:size(data, 2)
    fig.plot.layout["annotations"][i][:font][:size] = 20;
end

# Legend
if is_baseline == true

    # Size
    fig.plot.layout["width"]  = 800;
    fig.plot.layout["height"] = 1000;
    fig.plot.layout["legend"] = attr(orientation="h", y=-0.08, x=0.065, font=attr(size=14))

else
    # Size
    fig.plot.layout["width"]  = 1000;
    fig.plot.layout["height"] = 800;
    fig.plot.layout["legend"] = attr(orientation="h", y=-0.08, x=0.065, font=attr(size=14))
end

pltjs.savefig(fig, "$(model_folder)/img/historical_decomposition_fc.pdf", format="pdf")

"./models/baseline_iis//img/historical_decomposition_fc.pdf"

##### Trends

In [12]:
figures=Array{Any}(undef, 4)

for i=[1,2,3,4] .+ restricted_offset
    if i == 1 + restricted_offset
        trace0 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=data[3:3:last_observations[i], i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==4);
        trace1 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDPᵐ[1:3:last_observations[i], i] .+ T_GDPᵐ[2:3:last_observations[i], i] .+ T_GDPᵐ[3:3:last_observations[i], i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDP95[1:3:last_observations[i], i] .+ T_GDP95[2:3:last_observations[i], i] .+ T_GDP95[3:3:last_observations[i], i], name="CI, 90 pct.", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==4);
        trace3 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDP05[1:3:last_observations[i], i] .+ T_GDP05[2:3:last_observations[i], i] .+ T_GDP05[3:3:last_observations[i], i], name="CI, 90 pct.", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDP84[1:3:last_observations[i], i] .+ T_GDP84[2:3:last_observations[i], i] .+ T_GDP84[3:3:last_observations[i], i], name="CI, 84 pct.", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==4);
        trace5 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDP16[1:3:last_observations[i], i] .+ T_GDP16[2:3:last_observations[i], i] .+ T_GDP16[3:3:last_observations[i], i], name="CI, 84 pct.", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDPᵐ[1:3:last_observations[i], i] .+ T_GDPᵐ[2:3:last_observations[i], i] .+ T_GDPᵐ[3:3:last_observations[i], i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==4);


    elseif i == 2 + restricted_offset
        trace0 = pltjs.scatter(;x=date[3:3:end], y=data[3:3:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==4);
        trace1 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDPᵐ[3:3:last_observations[i], i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDP95[3:3:last_observations[i], i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==4);
        trace3 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDP05[3:3:last_observations[i], i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDP84[3:3:last_observations[i], i], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==4);
        trace5 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDP16[3:3:last_observations[i], i], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T_GDPᵐ[3:3:last_observations[i], i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==4);

    else
        trace0 = pltjs.scatter(;x=date[1:last_observations[i]], y=data[1:last_observations[i], i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==4);
        trace1 = pltjs.scatter(;x=date[1:last_observations[i]], y=iTᵐ[1:last_observations[i], i-1-restricted_offset], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[1:last_observations[i]], y=iT95[1:last_observations[i], i-1-restricted_offset], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==4);
        trace3 = pltjs.scatter(;x=date[1:last_observations[i]], y=iT05[1:last_observations[i], i-1-restricted_offset], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[1:last_observations[i]], y=iT84[1:last_observations[i], i-1-restricted_offset], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==4);
        trace5 = pltjs.scatter(;x=date[1:last_observations[i]], y=iT16[1:last_observations[i], i-1-restricted_offset], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[1:last_observations[i]], y=iTᵐ[1:last_observations[i], i-1-restricted_offset], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==4);
     end
    
     layout  = pltjs.Layout(;title=titles[i], titlefont=attr(size=12),
                            xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=-90, zeroline=false),
                            yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales[i]));
     if i != 3
         figures[i - restricted_offset] = pltjs.plot([trace1; trace2; 
                                                      trace1; trace3; 
                                                      trace1; trace4;
                                                      trace1; trace5; 
                                                      trace6; trace0], layout);
    else
         figures[i- restricted_offset] = pltjs.plot([trace1; trace2; 
                                                     trace1; trace3; 
                                                     trace1; trace4;
                                                     trace1; trace5; 
                                                     trace6; trace0], layout);
    end
end


fig = [figures[1] figures[2]; figures[3] figures[4]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 650;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:4
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.10, x=0.285, font=attr(size=10))

pltjs.savefig(fig, "$(model_folder)/img/trend_real.pdf", format="pdf")

"./models/baseline_iis//img/trend_real.pdf"

In [13]:
figures=Array{Any}(undef, 3)

for i=[6,7,8] .+ restricted_offset
    
     if i != 7 + restricted_offset
         trace0 = pltjs.scatter(;x=date[1:end], y=data[1:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==6);
     else
         trace0 = pltjs.scatter(;x=date[1:end], y=data[1:end, i], name="Data",mode="markers", marker=attr(size=3, color="black"), showlegend=i==6);
     end
    
    if i != 7 + restricted_offset
         trace0 = pltjs.scatter(;x=date[1:end], y=data[1:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==6);
         trace1 = pltjs.scatter(;x=date[1:last_observations[i]], y=Tᵐ[1:last_observations[i], i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
         trace2 = pltjs.scatter(;x=date[1:last_observations[i]], y=T95[1:last_observations[i], i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==6);
         trace3 = pltjs.scatter(;x=date[1:last_observations[i]], y=T05[1:last_observations[i], i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
         trace4 = pltjs.scatter(;x=date[1:last_observations[i]], y=T84[1:last_observations[i], i], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==6);
         trace5 = pltjs.scatter(;x=date[1:last_observations[i]], y=T16[1:last_observations[i], i], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
         trace6 = pltjs.scatter(;x=date[1:last_observations[i]], y=Tᵐ[1:last_observations[i], i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==6);

    else
        trace0 = pltjs.scatter(;x=date[3:3:end], y=data[3:3:end, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==6);
        trace1 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=Tᵐ[3:3:last_observations[i], i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T95[3:3:last_observations[i], i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==6);
        trace3 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T05[3:3:last_observations[i], i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T84[3:3:last_observations[i], i], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==6);
        trace5 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=T16[3:3:last_observations[i], i], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[3:3:last_observations[i]], y=Tᵐ[3:3:last_observations[i], i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==6);
     end
        
     layout  = pltjs.Layout(;title=titles[i], titlefont=attr(size=12),
                             xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=-90, zeroline=false, range=[date[1], date[last_observations[i]]]),
                             yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales[i], range=[-2,8]));
     
     figures[i-5-restricted_offset] = pltjs.plot([trace1; trace2; 
                                                  trace1; trace3; 
                                                  trace1; trace4;
                                                  trace1; trace5; 
                                                  trace6; trace0], layout);
end

fig = [figures[1]; figures[2]; figures[3]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:3
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.08, x=0.285, font=attr(size=10))

pltjs.savefig(fig, "$(model_folder)/img/trend_inflation.pdf", format="pdf")

"./models/baseline_iis//img/trend_inflation.pdf"

##### Output gap

In [14]:
# Compute output gap
output_gap = 100*(PC[1 + restricted_offset, :, :] .+ iC[1 + restricted_offset, :, :])./T_GDP[1 + restricted_offset, :, :];
output_gapᵐ = median(output_gap, dims=2)[:];

# Compute quartelry cycles and trends for GDP
quarterly_cycle = PC[1 + restricted_offset, 1:3:last_observations[1 + restricted_offset]+max_h, :] .+ PC[1 + restricted_offset, 2:3:last_observations[1 + restricted_offset]+max_h, :] .+ PC[1 + restricted_offset, 3:3:last_observations[1 + restricted_offset]+max_h, :] .+
                  iC[1 + restricted_offset, 1:3:last_observations[1 + restricted_offset]+max_h, :] .+ iC[1 + restricted_offset, 2:3:last_observations[1 + restricted_offset]+max_h, :] .+ iC[1 + restricted_offset, 3:3:last_observations[1 + restricted_offset]+max_h, :];

quarterly_trend = T_GDP[1 + restricted_offset, 1:3:last_observations[1 + restricted_offset]+max_h, :] .+ T_GDP[1 + restricted_offset, 2:3:last_observations[1 + restricted_offset]+max_h, :] .+ T_GDP[1 + restricted_offset, 3:3:last_observations[1 + restricted_offset]+max_h, :];

# Compute quarterly output gap
quarterly_output_gap  = 100*quarterly_cycle ./ quarterly_trend;
quarterly_output_gapᵐ = median(quarterly_output_gap, dims=2)[:];

In [15]:
figures = Array{Any}(undef,1)
trace1 = pltjs.scatter(;x=date[1:end], y=output_gapᵐ, line=attr(color="red"), name="Output gap");

layout = pltjs.Layout(;titlefont=attr(size=14), width=800, height=300)
fig = pltjs.plot(trace1, layout);
pltjs.savefig(fig, "$(model_folder)/img/output_gap_monthly.pdf", format="pdf")

"./models/baseline_iis//img/output_gap_monthly.pdf"

In [16]:
figures = Array{Any}(undef,1)
trace1 = pltjs.scatter(;x=date[3:3:end], y=quarterly_output_gapᵐ, line=attr(color="red"), name="Output gap (quarterly)");

layout = pltjs.Layout(;titlefont=attr(size=14), width=800, height=300)
fig = pltjs.plot(trace1, layout);
pltjs.savefig(fig, "$(model_folder)/img/output_gap_quarterly.pdf", format="pdf")

"./models/baseline_iis//img/output_gap_quarterly.pdf"

#### Key forecasts

##### Output

##### Inflation

### CSV output

In [17]:
using CSV;
CSV.write("$(model_folder)/results_csv/output_gap_quarterly.csv", DataFrame(x=quarterly_output_gapᵐ))
CSV.write("$(model_folder)/results_csv/output_gap_monthly.csv", DataFrame(x=output_gapᵐ))
CSV.write("$(model_folder)/results_csv/output_trend_quarterly.csv", DataFrame(x=σʸ[1 + restricted_offset] .* median(quarterly_trend, dims=2)[:]))
CSV.write("$(model_folder)/results_csv/output_trend_monthly.csv", DataFrame(x=T_GDPᵐ[:, 1 + restricted_offset]))
CSV.write("$(model_folder)/results_csv/inflation_trend_monthly.csv", DataFrame(x=Tᵐ[:,6 + restricted_offset]))

"./models/baseline_iis//results_csv/inflation_trend_monthly.csv"